In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

#packages for sending push notification
from pyfcm import FCMNotification
import json
import random

#packages for sending image
import requests
import uuid
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

sys.path.append("..")
#packages for visualizing
from utils import visualization_utils as vis_util
from utils import label_map_util
#packages for video capture
import cv2
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('http://131.163.210.9:8080/video')

#################################################################################################################
#person detection using tensorflow
def personDetection(): 
    sendNotificationOnce = False
    
    MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
    MODEL_FILE = MODEL_NAME + '.tar.gz'
    DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

    # Path to frozen detection graph. This is the actual model that is used for the object detection.
    PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

    # List of the strings that is used to add correct label for each box.
    PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

    NUM_CLASSES = 90

    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
        file_name = os.path.basename(file.name)
        if 'frozen_inference_graph.pb' in file_name:
            tar_file.extract(file, os.getcwd())

    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            while True:
                ret, image_np = cap.read()
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Each box represents a part of the image where a particular object was detected.
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Each score represent how level of confidence for each of the objects.
                # Score is shown on the result image, together with the class label.
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')

                # Actual detection.
                (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})

                boxes = np.squeeze(boxes)
                scores = np.squeeze(scores)
                classes = np.squeeze(classes)

                indices = np.argwhere(classes == 1)
                boxes = np.squeeze(boxes[indices])
                scores = np.squeeze(scores[indices])
                classes = np.squeeze(classes[indices])

                if(not sendNotificationOnce):
                    image_url = ""                  
                    image_url= uploadImage(image_np)
                    pushNotification(image_url)
                    sendNotificationOnce = True
                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                      image_np,
                      boxes,
                      classes.astype(np.int32),
                      scores,
                      category_index,
                      use_normalized_coordinates=True,
                      line_thickness=8)
                cv2.imshow('Person detection', cv2.resize(image_np, (800, 600)))
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    break

#send notification to Rescuers App
def pushNotification(image_url):
    push_service = FCMNotification(api_key="AAAAONGDDMk:APA91bFoj8Lv8Z5bf5SuZzKZZSTo6ulQlsMYX-LZAOenPS08UsWu9hSWEP1F4ZVHyMRC7-DVTuwqfc28floSUlezgtVkZBB6fTXj7SORvHxs-UDbqyBG6V3jDATaCiCJZfUYsoAAXggl")
 
    # Your api-key can be gotten from:  https://console.firebase.google.com/project/<project-name>/settings/cloudmessaging

    registration_id = "cZ7dnSlJivM:APA91bEDnG9qnIReM4OSC3yvEQG-rORwUvgWqXUr1w-PYuSScYxk1onJ2DVz2YBzRiB0OK93-5Q-WSryxX0hW7TxuUQhueoq17UIcU5c0_F85XMLau8x6fUHSOd7uv1aIkiPXGqmx-9E"
    message_title = "Rescuers"
    location = [
    "17.440081, 78.348915",
    "17.447412, 78.376228",
    "17.448294, 78.391487",
    "17.437462, 78.448288",
    "17.434719, 78.401138",
    "17.439930, 78.498276",
    "17.402963, 78.376705",
    "17.430613, 78.389960",
    "17.441061, 78.377338",
    ]
    message_body = '{"location": "'+random.choice(location) +'", "imageUrl": "'+ image_url+'"}'
    data_message = {"post_pk": message_body}

    print(message_body)
    result = push_service.notify_single_device(registration_id=registration_id, message_body=message_body, data_message=data_message)
    print(result)

#upload image to firebase
def uploadImage(image_np):
    #is_success, im_buf_arr = cv2.imencode(".jpg", image_np)
    is_success, im_buf_arr = cv2.imencode(".jpg", cv2.resize(image_np, (800, 600)))
    image_data = im_buf_arr.tobytes()
    #image_url = "https://www.gravatar.com/avatar/205e460b479e2e5b48aec07710c08d50" #we pass the url as an argument
    #image_data = requests.get(image_url).content
    if (not len(firebase_admin._apps)):
        cred = credentials.Certificate('./certificate2.json')
        firebase_admin.initialize_app(cred, {
            'storageBucket': 'ems-4-bce4c.appspot.com'
        })
    bucket = storage.bucket()
    filename = str(uuid.uuid4())
    blob = bucket.blob(filename+ '.jpg')
    blob.upload_from_string(
            image_data,
            content_type='image/jpg'
        )
    print(blob.public_url)
    return filename+ '.jpg'

#################################################################################################################

#call the person detection methods
personDetection()



C:\Users\pkyedidi\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\pkyedidi\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\pkyedidi\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\pkyedidi\AppData

https://storage.googleapis.com/ems-4-bce4c.appspot.com/02aa13c0-06a4-40f0-ba5f-cc50ace49c02.jpg
{"location": "17.437462, 78.448288", "imageUrl": "02aa13c0-06a4-40f0-ba5f-cc50ace49c02.jpg"}
{'multicast_ids': [1784308908337641424], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '0:1571318018629698%1a658da31a658da3'}], 'topic_message_id': None}
